# Libraries

In [ ]:
import os
import xarray as xr
import rioxarray as rxr
from shapely.geometry import mapping
import geopandas as gpd
from rasterio.enums import Resampling
from datetime import datetime
from calendar import monthrange

In [ ]:
# Set main directories for FAPAR data
main_dir = 'path_to_main_directory'
# Subdirectories for Copernicus data and boundary shape
tenDays_RT2 = os.path.join(main_dir, 'Copernicus', '10Days_RT2')
Boundary = os.path.join(main_dir, 'Boundary')
Output_dir = os.path.join(main_dir, 'Copernicus', 'Monthly_RT2')

In [ ]:
# Load and prepare shapefile
def load_shape_file(filepath):
    """Load shapefile to create a mask over the AOI."""
    shpfile = gpd.read_file(filepath)
    print("Shapefile loaded. To prepare for masking, run the function `select_shape`.")
    return shpfile

# Create mask from shapefile
def select_shape(shpfile):
    col_code = 'ISO3_CODE'
    country_codes = ['ZAF', 'LSO', 'SWZ']
    selected_rows = shpfile[shpfile[col_code].isin(country_codes)]
    unioned_polygon = selected_rows.geometry.unary_union
    mask_polygon = gpd.GeoDataFrame(geometry=[unioned_polygon])
    print("Mask created.")
    return mask_polygon

# Generate AOI
shpfile = load_shape_file(os.path.join(Boundary, 'boundary_shapefile.shp'))
AOI = select_shape(shpfile)

In [ ]:
# Processing FAPAR data by year and month
Copernicus_years = sorted(os.listdir(tenDays_RT2))
years_copernicus = {}
for year in Copernicus_years:
    months_copernicus = {}
    for month in sorted(os.listdir(os.path.join(tenDays_RT2, year))):
        FAPAR_10Days = sorted(os.listdir(os.path.join(tenDays_RT2, year, month)))
        path = os.path.join(tenDays_RT2, year, month)
        first_10Days = rxr.open_rasterio(os.path.join(path, FAPAR_10Days[0]), masked=True)
        second_10Days = rxr.open_rasterio(os.path.join(path, FAPAR_10Days[1]), masked=True)
        third_10Days = rxr.open_rasterio(os.path.join(path, FAPAR_10Days[2]), masked=True)

        Scale_factor = first_10Days.attrs['scale_factor']
        Number_of_Month = datetime.strptime(month, '%B').month
        nMonth_days = monthrange(int(year), Number_of_Month)[1]

        first_real_w = (first_10Days * Scale_factor) * (10 / nMonth_days)
        second_real_w = (second_10Days * Scale_factor) * (10 / nMonth_days)
        third_real_w = (third_10Days * Scale_factor) * ((nMonth_days + 1 - 21) / nMonth_days)
        concat = xr.concat([first_real_w, second_real_w, third_real_w], dim='band')
        Monthly_average = concat.sum(dim='band')
        Copernicus_crs = first_10Days.rio.crs
        Monthly_average.rio.write_crs(Copernicus_crs, inplace=True)

        Monthly_average_AOI = Monthly_average.rio.clip(AOI.geometry.apply(mapping),
                                   crs=Copernicus_crs, all_touched=True, from_disk=True).squeeze()

        months_copernicus[month] = Monthly_average_AOI
        check_folder_Output_dir = os.path.isdir(Output_dir)
        if not check_folder_Output_dir:
            os.makedirs(Output_dir)

        years_cop = os.path.join(Output_dir, year)
        check_folder_years_cop = os.path.isdir(years_cop)
        if not check_folder_years_cop:
            os.makedirs(years_cop)

        Monthly_average_AOI.rio.to_raster(os.path.join(years_cop, f'{year}_{month}.tif'))

    years_copernicus[year] = months_copernicus

print('Dictionaries of FAPAR data of all months per year are created.')